In [114]:
import os
import glob
from pathlib import Path
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline
plt.ion()   # interactive mode
from torch.utils.data import Dataset, DataLoader

In [115]:
class InputDataset(Dataset):
    def __init__(self, data_dir = 'data'):
        """
        Args: data_dir (string): data dir containing numeric and categorical csvs.
        """
        
        # check data_dir exists
        if not (Path(data_dir)).exists():
            raise FileNotFoundError('dir not found: {}'.format(data_dir))
        
        self.df = pd.DataFrame()
        
        self.filenames = []
        self.dfs = []
        
        self.read_csvs(data_dir + '/numeric')
        self.read_csvs(data_dir + '/categorical')
        
        self.merge_data()
    
    def read_csvs(self, data_dir):
        # check data_dir exists
        if not (Path(data_dir)).exists():
            raise FileNotFoundError('dir not found: {}'.format(data_dir))

        # read csvs into dataframes
        dfs = []
        names = []
        filenames = glob.glob(data_dir + '/*.csv')
        for f in filenames:
            df = pd.read_csv(f, header=None)
            self.dfs.append(df)
            self.filenames.append(f)
    
    def merge_data(self):
        pass
        
    def __len__(self):
        _len = 0
        for df in self.dfs:
            _len += len(df)
        return _len
    
    def __getitem__(self, idx):
        return None
        

In [119]:
ds = InputDataset()
print('len:', len(ds))
# for f, df in zip(ds.filenames, ds.dfs):
#     print(f)
#     print(df)


len: 15


In [124]:
df = pd.read_csv('data/numeric/n1.csv', header=None)
print(len(df))
df.iloc[0]

3


0      4.0
1      3.0
2      4.0
3      5.0
4      6.0
5      7.0
6      8.0
7      9.0
8     10.0
9     11.0
10    12.0
11    13.0
12    14.0
13    15.0
14    16.0
15    17.0
16    18.0
17    19.0
18    20.0
19    21.0
20    22.0
21    23.0
22    24.0
23    26.0
24    27.0
25     NaN
Name: 0, dtype: float64